In [148]:
import os
import csv
import xml.etree.ElementTree as et
import re

In [149]:
root_path = os.getcwd()
data_path = os.path.join(root_path,"filings")

In [150]:
def loadfiles():
    files = []
    for file in os.listdir(data_path):
        if file.endswith(".txt"):
            files.append(os.path.join(data_path, file))
    return files

In [151]:
files = loadfiles()

In [152]:
def childToDict(element, file_date):
    name = ""
    cusip = ""
    value = 0
    shares = 0
    for child in element.getchildren():
        xml_tag = re.sub( '\{.*\}','',child.tag, count=1)
        #print("element tag: ", xml_tag, " element value: ", child.text)
        if xml_tag == "nameOfIssuer":
            name = child.text
        if xml_tag == "cusip":
            cusip = child.text
        if xml_tag == "value":
            value = int(child.text)
        if xml_tag == "shrsOrPrnAmt":
            for grand_child in child.getchildren():
                gc_xml_tag = re.sub( '\{.*\}','',grand_child.tag, count=1)
                #print("     element tag: ", gc_xml_tag, " element value: ", grand_child.text)
                if gc_xml_tag == "sshPrnamt":
                    shares = int(grand_child.text)
    position_dict = {"file_date": file_date,  "name": name, "cusip": cusip, "mval": value, "shares": shares}
    #print(position_dict)
    return position_dict

In [153]:
def xmlToList(xml_str, file_date):
    position_list = []
    document = et.fromstring(xml_str)
    for element in document:
        position_list.append(childToDict(element, file_date))
    return position_list
        
                    

In [154]:
list_data = []
for file in files:
    file_name, ext = file[-14:].split('.')
    src = os.path.join(filepath, file)
    with open(src, 'r') as fh:
        output = str(fh.read())
        #print(output)
        list_data = list_data + xmlToList(output, file_name)

#print(list_data)

In [155]:
#print(list_data)
output = os.path.join(root_path,"sec13f_clean_data.csv")
with open(output, "w") as csvfile:
    field_names = ["file_date",  "name", "cusip", "mval", "shares"]
    csvwriter = csv.DictWriter(csvfile, lineterminator='\n', fieldnames=field_names)
    csvwriter.writeheader()
    csvwriter.writerows(list_data)